# Numeric backend interface

When using another backend value library, the operator precedence might alter how physipy and that package work together.
We don't get this problem with regular builtin and numpy instances, because int, float, list and so on do not know how to handle Quantities, so a NotImplemented is returned when they are on the left-hand-side of the operator. For example:

In [1]:
from physipy import m

# First 5.__mul__(m) is tried, but not implemented, so we fallback on 
# m.__rmul__(5) in physipy, where a conversion is builtin physipy to convert 5 
# into a Quantity, and 2 Quantities are easily multiplied together.
print(5 * m)

# Same applies for numpy:
import numpy as np
print(np.array([1,2,3]) * m)

5 m
[1 2 3] m


Let's see what happens with another custom class for illustration purpose: imagine we want to use a library that acts like numpy : with just little code, including : 
 - a conversion to Array in each operator with `arrayify`
 - `__array_priority__` > 0 so that `np.array([1,2]) * Array(5)` returns `Array([ 5 10])` and not `[Array(5) Array(10)]`

This way, we get a fully functionnal package that works with numpy.

In [2]:
def arrayify(x):
    if isinstance(x, Array):
        return x
    return Array(x)

HANDLED_FUNCTIONS = {}

class Array:

    def __iter__(self):
        return iter(self.array)

    def __abs__(self):
        return Array(abs(self.array))
    # __array_priority__ = 0  #  -> np.array([1,2]) * Array(5) = [Array(5) Array(10)]
    __array_priority__ = 1    #  -> np.array([1,2]) * Array(5) = Array([ 5 10])

    def __init__(self, array):
        self.array = array

    def __add__(self, other):
        other = arrayify(other)
        return Array(self.array + other.array)
    
    __radd__ = __add__

    def __repr__(self):
        return f"Array({self.array})"
        
    def __mul__(self, other):
        other = arrayify(other)
        return Array(self.array * other.array)
    
    __rmul__ = __mul__
    
    def __array_function__(self, func, types, args, kwargs):
        if func not in HANDLED_FUNCTIONS:
            return NotImplemented
        return HANDLED_FUNCTIONS[func](*args, **kwargs)


def implements(np_function):
    def decorator(func):
        HANDLED_FUNCTIONS[np_function] = func
        return func
    return decorator

@implements(np.sum)
def np_sum(q, **kwargs):
    return Array(np.sum(q.array))


print(Array(5) + np.array([1,2])) # Array([6 7])
print(Array(5) * np.array([1,2])) # Array([ 5 10]
print(np.array([1,2]) * Array(5)) # Array([ 5 10])
print(np.sum(Array(np.array([1,2,3]))))

Array([6 7])
Array([ 5 10])
Array([ 5 10])
Array(6)


Now let's see what happens if we use this package as the base value for physipy, so basicaly we will handle mostly Quantity with Array as values. First we define such a Quantity:

In [3]:
from physipy import Quantity, Dimension
x = Quantity(Array(np.array([1,-2,3])), Dimension('L'))
print(x)

Array([ 1 -2  3]) m


Now let's see how it behaves with regular operations. For the following, everything work as expected:

In [4]:
print(2*x)      # Array([2 4 6]) m
print(x*2)      # Array([2 4 6]) m
print(abs(2*x)) # Array([2 4 6]) m
print(x+3*m)    # Array([4 1 6]) m
print(3*m + x)  # Array([4 1 6]) m
print(np.sum(x))
print(m * Array(5))
print(m * Array(5) + 3*m)

Array([ 2 -4  6]) m
Array([ 2 -4  6]) m
Array([2 4 6]) m
Array([4 1 6]) m
Array([4 1 6]) m
Array(2) m
Array(5) m
Array(8) m


But what happens here : 

In [5]:
Array(5) * m

Array(5 m)

Since Array is on the left of the operator, its `__mul__` operator is called first, where it wraps the RHS into a Array, hence we get `Array(m)`, while we'd want `Array(1) m`. This happens because Array first converts any value that is not an Array isntance, into an Array that wraps this instance.

We cannot modify any physipy code to fix this, because everything happens in the Array class. So a solution is to modify the operators of Array so that they fail when use with a Quantity. The only difference with the above code is that the decorator `NotImplementedWithQuantity` is applied to the operators:

In [7]:
def NotImplementedWithQuantity(f):
    """Concept decorator for __op__ so that a backend class can defer to
    Quantity, even when the class instance is on the left of the operator.
    In the exeample below, if the decorator is not applied, then 
    Array(5) * m return Array(5 m) (an Array with a Quantity) instead of
    Array(5) m (a Quantity with an Array).
    """
    def new(*args, **kwargs):
        if any(isinstance(x, Quantity) for x in args):
            return NotImplemented
        else:
            return f(*args, **kwargs)
    return new


def arrayify(x):
    if isinstance(x, Array):
        return x
    return Array(x)

HANDLED_FUNCTIONS = {}

class Array:

    def __iter__(self):
        return iter(self.array)

    def __abs__(self):
        return Array(abs(self.array))
    # __array_priority__ = 0  #  -> np.array([1,2]) * Array(5) = [Array(5) Array(10)]
    __array_priority__ = 1    #  -> np.array([1,2]) * Array(5) = Array([ 5 10])

    def __init__(self, array):
        self.array = array

    @NotImplementedWithQuantity
    def __add__(self, other):
        other = arrayify(other)
        return Array(self.array + other.array)
    
    __radd__ = __add__

    def __repr__(self):
        return f"Array({self.array})"
        
    @NotImplementedWithQuantity
    def __mul__(self, other):
        other = arrayify(other)
        return Array(self.array * other.array)
    
    __rmul__ = __mul__
    
    def __array_function__(self, func, types, args, kwargs):
        if func not in HANDLED_FUNCTIONS:
            return NotImplemented
        return HANDLED_FUNCTIONS[func](*args, **kwargs)


def implements(np_function):
    def decorator(func):
        HANDLED_FUNCTIONS[np_function] = func
        return func
    return decorator

@implements(np.sum)
def np_sum(q, **kwargs):
    return Array(np.sum(q.array))


print(2*x)      # Array([2 4 6]) m
print(x*2)      # Array([2 4 6]) m
print(abs(2*x)) # Array([2 4 6]) m
print(x+3*m)    # Array([4 1 6]) m
print(3*m + x)  # Array([4 1 6]) m
print(np.sum(x))
print(m * Array(5))
print(m * Array(5) + 3*m)
print(Array(5)*m)

Array([ 2 -4  6]) m
Array([ 2 -4  6]) m
Array([2 4 6]) m
Array([4 1 6]) m
Array([4 1 6]) m
Array(2) m
Array(5) m
Array(8) m
Array(5) m


## Example wrapping uncertainties : add the unit to `nominal_value` and `std_dev`

Numpy's integration is a great example of how physipy can wrap any kind of numerical values, but this integration is written in the source code of physipy so it's a bit cheating.  
Let's see how physipy deals with a non-integrated numerical-like package : uncertainties. By "non-integrated" I mean that no source code of physipy makes any explicit reference to uncertainties, so we only rely on the general wrapping interface of physipy.

Let's first create a pure uncertainties variable

In [1]:
from physipy import m, s, K, Quantity, Dimension
import uncertainties.umath as um
import uncertainties as u

# create a pure uncertainties instance
x = u.ufloat(0.20, 0.01)  # x = 0.20+/-0.01
print(x)
print(x**2)
print(type(x))

0.200+/-0.010
0.040+/-0.004
<class 'uncertainties.core.Variable'>


If we create a quantity version by mulitplying by 1 meter, the returned value is of Quantity type:

In [3]:
# now let's create a quanity version of x
xq = x*m
print(xq)                       # a Quantity
print(xq**2, type(xq**2))       # a Quantity
print(xq+2*m, type(xq+2*m))     # a Quantity
print(xq.value, type(xq.value)) # an uncertainties value
print(m*x == x*m)               # True

0.200+/-0.010 m
0.040+/-0.004 m**2 <class 'physipy.quantity.quantity.Quantity'>
2.200+/-0.010 m <class 'physipy.quantity.quantity.Quantity'>
0.200+/-0.010 <class 'uncertainties.core.AffineScalarFunc'>
True


That's a pretty neat result __that didn't need any additional code__.  
Now going a bit further, uncertainties instance have a `nominal_value` and `std_dev` attributes.

In [4]:
# Creation must be done this way and not by "x*m" because "x*m" 
# will multiply the uncerainties variable by 1, and turn it into a
# AffineScalarFunc instance, which is not hashable and breaks my 
# register_property_backend that relies on dict lookup
#x = u.ufloat(0.20, 0.01)  # x = 0.20+/-0.01
xq = Quantity(x, Dimension("m")) # xq = x *m

In [5]:
print(x.nominal_value)
print(x.std_dev)

0.2
0.01


In physipy, if an attribute doesn't exist in the quantity instance, the lookup falls back on the backend value, ie on the uncertainties variable, so by default we get the same result on `xq` (note that we don't get auto-completion either for the same reason):

In [6]:
print(xq.nominal_value) # 0.2
print(xq.std_dev)       # 0.01

0.2
0.01


It would be great that `xq.nominal_value` actually prints `0.2 m`, not loosing the unit and making it explicit that the nominal value is actually 0.2 meters. To do that, we can add a property back to specify what we want `xq.nominal_value` to return : a property backend is a dictionnary with key the name of the attribute, and as value the corresponding method to get the wanted result.

For the nominal_value and standard deviation, we just want to add back the unit and make the variable a Quantity, so we multiply by the corresponding SI unit:

In [7]:
type(xq.value)

uncertainties.core.Variable

In [8]:
import uncertainties as uc
from physipy.quantity.quantity import register_property_backend

uncertainties_property_backend_interface = {
    # res is the backend result of the attribute lookup, and q the wrapping quantity
    "nominal_value":lambda q, res: q._SI_unitary_quantity*res,
    "std_dev":lambda q, res: q._SI_unitary_quantity*res,
    "n":lambda q, res: q._SI_unitary_quantity*res,
    "s":lambda q, res: q._SI_unitary_quantity*res,
}

register_property_backend(
    uc.core.Variable, 
    uncertainties_property_backend_interface
)

With this property back interface registered we get the desired result for `print(xq.nominal_value)`:

In [10]:
print(type(xq.value))
print(xq.nominal_value) # 0.2 m, instead of just 0.2 previously
print(xq.std_dev)       # 0.1 m, instead of just 0.1 previously

<class 'uncertainties.core.Variable'>
0.2 m
0.01 m


## Using subclass to extend Quantity's support

Another approach to do this would be to create a new class like this

In [11]:
from physipy import m, s, K, Quantity, Dimension
import uncertainties as u

# create a pure uncertainties instance
x = u.ufloat(0.20, 0.01)  # x = 0.20+/-0.01

class UWrappedQuantity(Quantity):
    @property
    def nominal_value(self):
        return self.value.nominal_value * self._SI_unitary_quantity
    @property
    def std_dev(self):
        return self.value.std_dev * self._SI_unitary_quantity

xq2 = UWrappedQuantity(x, Dimension("m"))
#print(xq2)
print(type(xq2), xq2, xq2.nominal_value)

print(xq2+2*m, type(xq2+2*m), (xq2+2*m).value, (xq2+2*m)._SI_unitary_quantity)
print((xq2+2*m).nominal_value)

<class '__main__.UWrappedQuantity'> 0.200+/-0.010 m 0.2 m
2.200+/-0.010 m <class '__main__.UWrappedQuantity'> 2.200+/-0.010 1 m
2.2 m


## Interface

The simplest way to use physipy with specific objects, like fractions or your own class is to create quantities that wrap your value : here `Quantity` wraps the custom value as its `value` attribute.

A simple example : 

In [8]:
import fractions
from physipy import Quantity, Dimension, m

In [9]:
# from constructor
length = Quantity(fractions.Fraction(3, 26), Dimension("m"))
# or altneratively
length = fractions.Fraction(3, 26) * m
print(length)

3/26 m


Then when doing calculation, physipy deals with everything for you :

In [10]:
print(length + 2*m)
print(length**2)
print(length.dimension)
print(length.is_mass())
print(length.sum())

55/26 m
9/676 m**2
L
False
3/26 m


Note that there are unittests based on fractions.Fraction.

# More complex objects

Now say we want to customize the basic Fraction object, by overloading its str method and work only with this new class:

In [26]:
class MyFraction(fractions.Fraction):
    def __str__(self):
        return f"[[[{self.numerator}/{self.denominator}]]]"

In [31]:
my_length = MyFraction(3, 26)
print(my_length)
print(my_length*2)
print(2*my_length)
print(Quantity(MyFraction(3, 26), Dimension("L")))
print(MyFraction(3, 26)*m)
print(m*MyFraction(3, 26))

[[[3/26]]]
3/13
3/13
[[[3/26]]] m
3/26 m
3/26 m


Notice that we lost the custom str : that's because the value is not a `MyFraction` instance but `fractions.Fraction` : it was lost in the multiplication process. Indeed, since `my_length*m` falls back on the `__mul__` of Quantity, that uses the `__mul__` method of the instance, which is `fractions.Fraction.__mul__`, which returns a `fractions.Fraction` instance, not a `MyFraction` instance.

So we would like physipy to return a `MyFraction` when computing multiplication with a Quantity objet.

Now we can't expect each user to rewrite its custom Fraction class to be compatible with Quantity, so we do the opposite : Quantity will wrap the custom class with an interface class.

In [32]:
class QuantityWrappedMyFraction(Quantity):
    def __str__(self):
        str_myfraction = str(self.value)
        return "QuantityWrappedMyFraction : " + str_myfraction + "-"+self.dimension.str_SI_unit()

from physipy.quantity.quantity import register_value_backend
register_value_backend(
    # here we use the class of the base value
    fractions.Fraction, 
    # here the rewritten version
    QuantityWrappedMyFraction)

ImportError: cannot import name 'register_value_backend' from 'physipy.quantity.quantity' (C:\Users\ym\Documents\REPOS\physipy\physipy\quantity\quantity.py)

In [33]:
print(my_length)
print(my_length*my_length)
print(Quantity(MyFraction(3, 26), Dimension("L")))
print(MyFraction(3, 26)*m)

[[[3/26]]]
9/676
[[[3/26]]] m
3/26 m


# Requirements for easy backend supports

 - The class must be hashable : for the dict lookup on type, we need the class as key, so they need to be hashable